In [57]:

#set year: 2023, 24, 25
year = 2023

In [58]:
import pandas as pd

#we load the list of distinct tpurnaments per year.
#We got this from SQL query at "query_tournaments"
df_tornei = pd.read_csv(f"tornei/tornei_{str(year)[-2:]}.csv")
df_iscrizioni = pd.read_csv(f"df_iscrizioni_player_event/df_{year}.csv")

players = df_iscrizioni["PlayerCode"].dropna().unique()
events  = df_tornei["EventId"].dropna().unique()

grid = (    pd.MultiIndex.from_product([players, events], names=["PlayerCode", "EventId"]).to_frame(index=False))

iscr_key = df_iscrizioni[["PlayerCode", "EventId","PointsGained"]].drop_duplicates()
iscr_key["iscritto"] = 1
out = grid.merge(iscr_key, on=["PlayerCode", "EventId"], how="left")
out["iscritto"] = out["iscritto"].fillna(0).astype(int)

out["PointsGained"] = out["PointsGained"].fillna(0).astype(int)


out = out.merge(df_tornei, on="EventId", how="left")

display(out)


,PlayerCode,EventId,PointsGained,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney
0,mv14,319,45,1,2023,2023-07-31,Kitzbuhel,250,Austria,Clay,562815
1,mv14,414,0,1,2023,2023-07-24,Hamburg,500,Germany,Clay,1831515
2,mv14,316,0,1,2023,2023-07-17,Bastad,250,Sweden,Clay,562815
3,mv14,2841,90,1,2023,2023-04-17,Banja Luka,250,Bosnia & Herzegovina,Clay,562815
4,mv14,807,0,1,2023,2023-02-27,Acapulco,500,Mexico,Hard,2013940
...,...,...,...,...,...,...,...,...,...,...,...
18355,m0bn,499,0,0,2023,2023-02-13,Delray Beach,250,"FL, U.S.A.",Hard,642735
18356,m0bn,424,0,0,2023,2023-02-06,Dallas,250,"TX, U.S.A.",Hard,737170
18357,m0bn,747,0,0,2023,2023-09-25,Beijing,500,China,Hard,3633875
18358,m0bn,311,0,0,2023,2023-06-19,London / Queen's Club,500,Great Britain,Grass,2195175


In [59]:
#we add a column that tells if the player's nationality is the same as the tournament country
#tornei = pd.read_csv(f"df_iscrizioni_player_event/df_{year}.csv")
players = pd.read_csv(f"players_list/atp_senza_ripetizioni_{year}.csv")

tornei = out.copy()
tornei = tornei[(tornei["EventType"] == 250) | (tornei["EventType"] == 500)]

# if EventCountry ends with "U.S.A." replace it with "USA"
tornei["EventCountry"] = tornei["EventCountry"].str.replace(r".*U\.S\.A\.$", "United States of America", regex=True)
display(tornei["EventCountry"].unique())
tornei = tornei[["PlayerCode", "EventId", "EventCountry"]]

players = players[["Player Id", "Nationality"]]

df = tornei.merge(
    players,
    left_on="PlayerCode",
    right_on="Player Id",
    how="left"
)

#we load the geo data for country codes from a github repo: "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv"
#this because the players' nationalities are in alpha-3 code, while the tournament countries are in full name
geo1 = pd.read_csv("geografia.csv")

df_primo_join = df.merge(geo1,
    left_on="EventCountry",
    right_on="name",
    how="left"
)

df_primo_join["Nationality"] = df_primo_join["Nationality"].str.upper()


df_finale = df_primo_join[df_primo_join["Nationality"] == df_primo_join["alpha-3"]]
df_finale=df_finale[["PlayerCode", "EventId", "EventCountry","Nationality"]]
df_finale

array(['Austria', 'Germany', 'Sweden', 'Bosnia & Herzegovina', 'Mexico',
       'Brazil', 'Argentina', 'New Zealand', 'India', 'Kazakhstan',
       'China', 'United States of America', 'Spain', 'France', 'Belgium',
       'Netherlands', 'Switzerland', 'U.A.E.', 'Portugal', 'Chile',
       'Bulgaria', 'Croatia', 'Great Britain', 'Australia', 'Morocco',
       'Japan', 'Qatar'], dtype=object)

,PlayerCode,EventId,EventCountry,Nationality
68,mw02,7694,France,FRA
74,mw02,496,France,FRA
78,mw02,375,France,FRA
79,mw02,341,France,FRA
119,mc65,7694,France,FRA
...,...,...,...,...
18067,m0nb,315,United States of America,USA
18088,m0nb,6242,United States of America,USA
18099,m0nb,717,United States of America,USA
18100,m0nb,499,United States of America,USA


In [60]:
# add column same_n
out["Same_Nationality"] = 0

for index, row in out.iterrows():
    player_code = row["PlayerCode"]
    event_id = row["EventId"]
    
    match = df_finale[
        (df_finale["PlayerCode"] == player_code) & 
        (df_finale["EventId"] == event_id)
    ]
    
    if not match.empty:
        out.at[index, "Same_Nationality"] = 1
out

,PlayerCode,EventId,PointsGained,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney,Same_Nationality
0,mv14,319,45,1,2023,2023-07-31,Kitzbuhel,250,Austria,Clay,562815,0
1,mv14,414,0,1,2023,2023-07-24,Hamburg,500,Germany,Clay,1831515,0
2,mv14,316,0,1,2023,2023-07-17,Bastad,250,Sweden,Clay,562815,0
3,mv14,2841,90,1,2023,2023-04-17,Banja Luka,250,Bosnia & Herzegovina,Clay,562815,0
4,mv14,807,0,1,2023,2023-02-27,Acapulco,500,Mexico,Hard,2013940,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18355,m0bn,499,0,0,2023,2023-02-13,Delray Beach,250,"FL, U.S.A.",Hard,642735,0
18356,m0bn,424,0,0,2023,2023-02-06,Dallas,250,"TX, U.S.A.",Hard,737170,0
18357,m0bn,747,0,0,2023,2023-09-25,Beijing,500,China,Hard,3633875,0
18358,m0bn,311,0,0,2023,2023-06-19,London / Queen's Club,500,Great Britain,Grass,2195175,0


In [61]:
out.to_csv(f"df_per_aggiungere_pts_def/df_{year}.csv", index=False)

#binary variable PTS_to_defend is then added with a SQL query at "query_aggiunti_pts_def"
#the resulting df are stored in "df_pre_dummies" folder with the final df used for regression.